In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

%matplotlib inline
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
from sklearn.feature_selection import RFE
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
df = pd.read_csv("C:/python/hamoye/Machine Learning Regression - Predicting Energy Efficiency of Buildings/energydata_complete.csv")

In [3]:
df.sample()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
18774,2016-05-21 02:00:00,60,0,23.066667,45.526667,21.39,47.23,24.89,42.26,23.2,...,22.6,46.42,14.3,757.2,90.0,4.0,40.0,12.6,25.699484,25.699484


In [4]:
x = df.T2
y = df.T6
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3,random_state=0)

In [5]:
x_train.shape,y_train.shape, x_test.shape, y_test.shape

((13814,), (13814,), (5921,), (5921,))

In [6]:
linear_model = LinearRegression()
linear_model.fit(x_train,y_train)
predicted_values = linear_model.predict(x_test)
predicted_values

ValueError: Expected 2D array, got 1D array instead:
array=[22.32333333 21.         21.35666667 ... 19.         20.79
 20.6       ].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [7]:
x_train.shape,y_train.shape, x_test.shape, y_test.shape

((13814,), (13814,), (5921,), (5921,))

In [8]:
df1 = df.drop(['date','lights'],axis=1)
scaler = MinMaxScaler()
normalisedDf = pd.DataFrame(scaler.fit_transform(df1),
                           columns=df1.columns)
y = normalisedDf['Appliances']

In [9]:
x_train,x_test,y_train,y_test = train_test_split(normalisedDf,y,test_size=0.3, random_state=42)

In [10]:
linear_model = LinearRegression()
linear_model.fit(x_train,y_train)
predicted_values = linear_model.predict(x_test)
predicted_values

array([0.02803738, 0.07476636, 0.03738318, ..., 0.03738318, 0.09345794,
       0.03738318])

In [11]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test,predicted_values)
mae

1.163576517929915e-16

In [12]:
round(mae, 3)

0.0

In [13]:
rmse = np.sqrt(mean_squared_error(y_test,predicted_values))
rmse

1.8896605646284427e-16

In [14]:
round(rmse,3)

0.0

In [15]:
r2 = r2_score(y_test,predicted_values)
round(r2,3)

1.0

In [16]:
ridge_reg = Ridge(alpha=0.5)
ridge_reg.fit(x_train,y_train)
lasso_reg = Lasso(alpha=0.001)
lasso_reg.fit(x_train,y_train)

Lasso(alpha=0.001)

In [17]:
def get_weights_df(model,feat,col_name):
    #this fnx returns the weight of every feature
    weights = pd.Series(model.coef_,feat.columns).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = ['Features', col_name]
    weights_df[col_name].round(3)
    return weights_df

linear_model_weights = get_weights_df(linear_model, x_train, 'Linear_Model_Weight')
ridge_weights_df = get_weights_df(ridge_reg, x_train,'Ridge_weight')
lasso_weights_df =get_weights_df(lasso_reg, x_train, 'Lasso_weight')

final_weights = pd.merge(linear_model_weights, ridge_weights_df, on='Features')
final_weights = pd.merge(final_weights, lasso_weights_df, on='Features')

In [18]:
final_weights

,Features,Linear_Model_Weight,Ridge_weight,Lasso_weight
0,RH_8,-1.553397e-16,-0.000710,-0.000000
1,T2,-1.552275e-16,-0.000879,0.000000
2,Windspeed,-1.530044e-16,0.000138,0.000000
3,RH_7,-9.834812e-17,-0.000210,-0.000000
4,T7,-9.200157e-17,0.000045,0.000000
5,T9,-6.952158e-17,-0.000855,0.000000
6,T4,-5.152382e-17,0.000122,0.000000
7,Visibility,-4.882938e-17,0.000055,0.000000
8,rv2,-4.829524e-17,0.000003,-0.000000
9,rv1,-4.829524e-17,0.000003,-0.000000


In [19]:
lasso_reg = Lasso(alpha=0.001)
lasso_reg.fit(x_train,y_train)

Lasso(alpha=0.001)